In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
import pandas as pd
import numpy as np

import tensorflow.keras as K
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as bk
# import tensorflow_addons as tfa

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pickle
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# ! cp './drive/MyDrive/Wipro/Data/memotion1_images_combined.zip' .
# ! unzip -q memotion1_images_combined.zip
# ! rm -rf memotion1_images_combined.zip

! cp './drive/MyDrive/Wipro/Final/Data/Memotion/images.zip' .
! unzip -q images.zip
! rm -rf images.zip

In [ ]:
! cp './drive/MyDrive/Wipro/Final/Data/Memotion/pickles/memotion_img_embedding.pkl' .
! cp './drive/MyDrive/Wipro/Final/Data/Memotion/pickles/memotion_text_embedding.pkl' .

## Data Preparation

### Word Level Data

### Extra 

In [ ]:
df_merged = pd.read_csv('./drive/MyDrive/Wipro/Data/merged_data.csv')

In [ ]:
df_merged.head()

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,tagged_sentence
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,positive,look friend lightyear sohalikut trend play 10 ...
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,positive,best 10 yearchallenge
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive,sam thorne @ strippin follow follow saw everyo...
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive,10 year challenge sweet dee edition
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral,10 year challenge filter 47 hilarious 10 year ...


In [ ]:
df_word = pd.read_csv('./drive/MyDrive/Wipro/Data/image_word_pairs.csv')

In [ ]:
df_word.head()

In [ ]:
df_merged.shape,df_word.shape

In [ ]:
df_merged['image_name'].unique().shape

In [ ]:
df_word['image_name'].unique().shape

In [ ]:



rem_images = ['image_1567.jpg','image_4924.jpg','image_5119.png','image_6357.jpg']
df_word_cleaned = df_word.drop(df_word[df_word['image_name'].isin(rem_images)].index)

# df['overall_sentiment'] = df['overall_sentiment'].apply(get_labels)

df_word_cleaned.dropna(inplace=True)

In [ ]:
df_word_cleaned.shape

In [ ]:
df_word_cleaned['image_name'].unique().shape

In [ ]:
def apply_humour(label):

  if label == 'not_funny':
    return 0

  elif label == 'funny':
    return 1

  elif label == 'very_funny':
    return 2

  else: 
    return 3


In [ ]:
def apply_sarcasm(label):

  if label == 'not_sarcastic':
    return 0

  elif label == 'general':
    return 1

  elif label == 'twisted_meaning':
    return 2
    
  else:
    return 3
    

In [ ]:
def apply_offensive(label):

  if label == 'not_offensive':
    return 0

  elif label == 'slight':
    return 1

  elif label == 'very_offensive':
    return 2
    
  else:
    return 3


In [ ]:
def apply_motivational(label):

  if label == 'not_motivational':
    return 0
    
  else:
    return 1



In [ ]:
df_sentence = df_merged.copy()

df_sentence['humour'] = df_sentence['humour'].apply(apply_humour)
df_sentence['sarcasm'] = df_sentence['sarcasm'].apply(apply_sarcasm)
df_sentence['offensive'] = df_sentence['offensive'].apply(apply_offensive)
df_sentence['motivational'] = df_sentence['motivational'].apply(apply_motivational)

In [ ]:
df_sentence.shape

In [ ]:
df_sentence.tail()

In [ ]:


humour_labels = []
sarcasm_labels = []
offensive_labels = []
motivational_labels = []

for img_name in df_word_cleaned['image_name']:

  labels = df_sentence[df_sentence['image_name'] == img_name][['humour','sarcasm','offensive','motivational']]

  humour_labels.append(labels['humour'].values[0])
  sarcasm_labels.append(labels['sarcasm'].values[0])
  offensive_labels.append(labels['offensive'].values[0])
  motivational_labels.append(labels['motivational'].values[0])


  # if labels.shape[0]!=1:
  #   print(img_name)

In [ ]:
humour_series = pd.Series(humour_labels, dtype=int)
sarcasm_series = pd.Series(sarcasm_labels, dtype=int)
offensive_series = pd.Series(offensive_labels, dtype=int)
motivational_series = pd.Series(motivational_labels, dtype=int)

In [ ]:
df_word_cleaned['humour'] = humour_series
df_word_cleaned['sarcasm'] = sarcasm_series
df_word_cleaned['offensive'] = offensive_series
df_word_cleaned['motivational'] = motivational_series

In [ ]:
df_train, df_val = train_test_split(df_word_cleaned,test_size=0.2)
df_test,df_val = train_test_split(df_val,test_size=0.5)

### Dataset

In [ ]:
with open('memotion_img_embedding.pkl','rb') as f:
  img_embed_pkl = pickle.load(f)

with open('memotion_text_embedding.pkl','rb') as f:
  text_embed_pkl = pickle.load(f)

In [ ]:
def ret_img_word_embeddings(images, words):

  img_embeddings = []
  word_embeddings = []

  for (x,y) in  zip(images,words):

    # img_name, word = df_train.iloc[i]['image_name'], df_train.iloc[i]['word']

    img_embeddings.append(img_embed_pkl[x])
    word_embeddings.append(text_embed_pkl[y])


  np_img_embeddings_train = np.array(img_embeddings)
  np_word_embeddings_train = np.array(word_embeddings)

  return np_img_embeddings_train, np_word_embeddings_train

# labels_temp_train = df_train.dropna()[['humour','sarcasm','offensive','motivational']].values

In [ ]:
# img_embeddings = []
# word_embeddings = []

# for (x,y) in  df_val.dropna()[['image_name','word']].values:

#   # img_name, word = df_val.iloc[i]['image_name'], df_val.iloc[i]['word']

#   img_embeddings.append(img_embed_pkl[x])
#   word_embeddings.append(text_embed_pkl[y])


# np_img_embeddings_val = np.array(img_embeddings)
# np_word_embeddings_val = np.array(word_embeddings)


# labels_temp_val = df_val.dropna()[['humour','sarcasm','offensive','motivational']].values

In [ ]:
# img_embeddings = []
# word_embeddings = []

# for (x,y) in  df_test.dropna()[['image_name','word']].values:

#   # img_name, word = df_test.iloc[i]['image_name'], df_test.iloc[i]['word']

#   img_embeddings.append(img_embed_pkl[x])
#   word_embeddings.append(text_embed_pkl[y])


# np_img_embeddings_test = np.array(img_embeddings)
# np_word_embeddings_test = np.array(word_embeddings)


# labels_temp_test = df_test.dropna()[['humour','sarcasm','offensive','motivational']].values

In [ ]:

# df = pd.read_csv('./drive/MyDrive/Wipro/Data/memotion_words.csv')

df = pd.read_csv('/content/drive/MyDrive/Wipro/Final/Data/Memotion/csv/memotion_train_words.csv')
# df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

,image_name,word,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1336.jpg,friend,2,2,2,0,2
1,image_850.png,times,1,1,1,1,2
2,image_6032.jpg,ask,2,2,2,0,2
3,image_5589.jpg,recognition,1,2,2,1,2
4,image_6691.png,imagine,2,2,2,0,2


In [ ]:

# Remove Images that don't exist

rem_images = ['image_1567.jpg','image_4924.jpg','image_5119.png','image_6357.jpg']
df_word_cleaned = df.drop(df[df['image_name'].isin(rem_images)].index)
df_word_cleaned.dropna(inplace=True)
df_word_cleaned.index = range(df_word_cleaned.shape[0])


In [ ]:
df_word_cleaned.head()

,image_name,word,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1336.jpg,friend,2,2,2,0,2
1,image_850.png,times,1,1,1,1,2
2,image_6032.jpg,ask,2,2,2,0,2
3,image_5589.jpg,recognition,1,2,2,1,2
4,image_6691.png,imagine,2,2,2,0,2


In [ ]:
label_columns = ['humour','sarcasm','offensive','motivational']

In [ ]:
replace_dict = {
    'humour' : { 'not_funny': 0, 'funny' :1 , 'very_funny':2, 'hilarious': 3}  ,
    'sarcasm' : {  'not_sarcastic':0, 'general':1, 'twisted_meaning':2, 'very_twisted':3  } ,
    'offensive' :{  'not_offensive':0, 'slight':1, 'very_offensive':2, 'hateful_offensive':3  } ,
    'motivational' :{  'not_motivational':0, 'motivational':1},
}

In [ ]:
df_replaced = df_word_cleaned.replace(replace_dict)

In [ ]:
def create_one_hot_df(original_df, data, columns):

  enc = OneHotEncoder(sparse=False)
  arr = enc.fit_transform(data.values.reshape(-1,1))

  one_hot_df = pd.DataFrame( arr , dtype= np.uint8, columns = columns)

  final_df = pd.concat([original_df, one_hot_df] , axis=1)

  return final_df


In [ ]:
h_columns = ['not_funny', 'funny', 'very_funny','hilarious' ]
s_columns = ['not_sarcastic', 'general', 'twisted_meaning','very_twisted' ]
o_columns = ['not_offensive', 'slight', 'very_offensive','hateful_offensive' ]
m_columns = ['not_motivational', 'motivational']


df_numerical = create_one_hot_df(df_replaced, df_replaced['humour'], columns= h_columns)
df_numerical = create_one_hot_df(df_numerical, df_replaced['sarcasm'], columns= s_columns)
df_numerical = create_one_hot_df(df_numerical, df_replaced['offensive'], columns= o_columns)
# df_numerical = create_one_hot_df(df_numerical, df_replaced['motivational'], columns= m_columns)


In [ ]:
df_numerical.loc[:, 'not_funny': 'hilarious']

,not_funny,funny,very_funny,hilarious
0,1,0,0,0
1,0,0,0,1
2,1,0,0,0
3,0,0,0,1
4,0,1,0,0
...,...,...,...,...
58210,0,0,1,0
58211,0,0,1,0
58212,1,0,0,0
58213,0,0,0,1


In [ ]:
df_numerical

,image_name,word,overall_sentiment,humour,sarcasm,offensive,motivational,not_funny,funny,very_funny,hilarious,not_sarcastic,general,twisted_meaning,very_twisted,not_offensive,slight,very_offensive,hateful_offensive
0,image_4737.png,films,positive,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0
1,image_6131.png,2:11,positive,3,2,1,0,0,0,0,1,0,0,1,0,0,1,0,0
2,image_4709.jpg,spiderman,positive,0,3,2,0,1,0,0,0,0,0,0,1,0,0,1,0
3,image_2842.png,morpheus,positive,3,3,3,0,0,0,0,1,0,0,0,1,0,0,0,1
4,image_6734.jpg,picard,positive,1,2,1,0,0,1,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58210,racis_31437070548.jpg,goes,negative,2,2,1,0,0,0,1,0,0,0,1,0,0,1,0,0
58211,image_5391.jpg,last,positive,2,1,2,0,0,0,1,0,0,1,0,0,0,0,1,0
58212,image_55.jpg,'re,positive,0,3,2,1,1,0,0,0,0,0,0,1,0,0,1,0
58213,image_5163.png,walk,positive,3,2,1,0,0,0,0,1,0,0,1,0,0,1,0,0


In [ ]:
'''
  Train, Val, Test Split ( 80-10-10)
'''

df_train,df_val = train_test_split(df_word_cleaned,test_size=0.2)
# df_val,df_test = train_test_split(df_test,test_size=0.5)

In [ ]:
df_train.shape, df_val.shape

((37939, 7), (9485, 7))

In [ ]:
# train_dataset_img = tf.data.Dataset.from_tensor_slices(np.squeeze(np_img_embeddings_train,1))
# train_dataset_text = tf.data.Dataset.from_tensor_slices(np.squeeze(np_word_embeddings_train,1))
# train_dataset_labels = tf.data.Dataset.from_tensor_slices(labels_temp_train)

# train_dataset = tf.data.Dataset.zip((train_dataset_img, train_dataset_text, train_dataset_labels))

In [ ]:
# val_dataset_img = tf.data.Dataset.from_tensor_slices(np.squeeze(np_img_embeddings_val,1))
# val_dataset_text = tf.data.Dataset.from_tensor_slices(np.squeeze(np_word_embeddings_val,1))
# val_dataset_labels = tf.data.Dataset.from_tensor_slices(labels_temp_val)

# val_dataset = tf.data.Dataset.zip((val_dataset_img, val_dataset_text, val_dataset_labels))

In [ ]:
# test_dataset_img = tf.data.Dataset.from_tensor_slices(np.squeeze(np_img_embeddings_test,1))
# test_dataset_text = tf.data.Dataset.from_tensor_slices(np.squeeze(np_word_embeddings_test,1))
# test_dataset_labels = tf.data.Dataset.from_tensor_slices(labels_temp_test)

# test_dataset = tf.data.Dataset.zip((test_dataset_img, test_dataset_text, test_dataset_labels))

In [ ]:
# train_dataset_img = tf.data.Dataset.from_tensor_slices(np.squeeze(np_img_embeddings_train,1))
# train_dataset_text = tf.data.Dataset.from_tensor_slices(np.squeeze(np_word_embeddings_train,1))
# train_dataset_labels = tf.data.Dataset.from_tensor_slices(labels_temp_train)

In [ ]:
def ret_dataset(df):

  dataset = tf.data.Dataset.from_tensor_slices(
    (
        df['image_name'].values,
        df['word'].values,
        df['humour'].values,
        df['sarcasm'].values,
        df['offensive'].values,
        df['motivational'].values,
    )  
  )

  return dataset


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
  (
      df_train['image_name'].values,
      df_train['word'].values,
      df_train['humour'].values,
      df_train['sarcasm'].values,
      df_train['offensive'].values,
      df_train['motivational'].values,
  )  
)

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(
  (
      df_val['image_name'].values,
      df_val['word'].values,
      df_val['humour'].values,
      df_val['sarcasm'].values,
      df_val['offensive'].values,
      df_val['motivational'].values,
  )  
)

In [ ]:
split_dataset = tf.data.Dataset.from_tensor_slices(
  (
      df_word_cleaned['image_name'].values,
      df_word_cleaned['word'].values,
      df_word_cleaned['humour'].values,
      df_word_cleaned['sarcasm'].values,
      df_word_cleaned['offensive'].values,
      df_word_cleaned['motivational'].values,
  )  
)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37939 entries, 0 to 47423
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_name         37939 non-null  object
 1   word               37936 non-null  object
 2   humour             37939 non-null  int64 
 3   sarcasm            37939 non-null  int64 
 4   offensive          37939 non-null  int64 
 5   motivational       37939 non-null  int64 
 6   overall_sentiment  37939 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 2.3+ MB


In [ ]:
df_word_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47424 entries, 0 to 47423
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   image_name         47424 non-null  object
 1   word               47424 non-null  object
 2   humour             47424 non-null  int64 
 3   sarcasm            47424 non-null  int64 
 4   offensive          47424 non-null  int64 
 5   motivational       47424 non-null  int64 
 6   overall_sentiment  47424 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 2.5+ MB


In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=21)

In [ ]:
for train_idx, test_idx in kfold.split(df_word_cleaned):
  # df_train = df_word_cleaned.iloc
  print(train_idx.shape, test_idx.shape)
  df_train = df.iloc[train_idx]
  df_val = df.iloc[test_idx]

  print(df_train.shape)
  print(df_val.shape)

  break

(37939,) (9485,)
(37939, 7)
(9485, 7)


In [ ]:
df_train.head()

,image_name,word,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1336.jpg,friend,2,2,2,0,2
1,image_850.png,times,1,1,1,1,2
3,image_5589.jpg,recognition,1,2,2,1,2
4,image_6691.png,imagine,2,2,2,0,2
5,image_3404.jpeg,accisememes,2,0,1,0,2


In [ ]:
df_val.head()

,image_name,word,humour,sarcasm,offensive,motivational,overall_sentiment
2,image_6032.jpg,ask,2,2,2,0,2
7,image_3550.jpg,tries,1,2,1,0,0
21,image_6662.png,biggus,2,0,0,0,2
22,image_5090.jpg,imgflip.com,2,2,1,1,2
24,image_5799.jpg,ipad,1,1,0,0,2


In [ ]:
train_idx

array([    0,     1,     3, ..., 47419, 47421, 47423])

In [ ]:
df_test = pd.read_csv('./drive/MyDrive/Wipro/Final/Data/Memotion/csv/memotion_test_words.csv')
df_test.dropna(inplace=True)

In [ ]:
df_test.shape

(12888, 7)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(
  (
      df_test['image_name'].values,
      df_test['word'].values,
      df_test['humour'].values,
      df_test['sarcasm'].values,
      df_test['offensive'].values,
      df_test['motivational'].values,
  )  
)

In [ ]:

class_weights_h = list(compute_class_weight('balanced', np.unique(df_word_cleaned['humour'].values), df_word_cleaned['humour'].values))
class_weights_s = list(compute_class_weight('balanced', np.unique(df_word_cleaned['sarcasm'].values), df_word_cleaned['sarcasm'].values))
class_weights_o = list(compute_class_weight('balanced', np.unique(df_word_cleaned['offensive'].values), df_word_cleaned['offensive'].values))
class_weights_m = list(compute_class_weight('balanced', np.unique(df_word_cleaned['motivational'].values), df_word_cleaned['motivational'].values))

In [ ]:
class_weights_h, class_weights_s, class_weights_o, class_weights_m

([1.016635225518779,
  0.7250489236790607,
  0.7836087243886318,
  2.770093457943925],
 [1.1126126126126126,
  0.5056079150496823,
  1.108762741980735,
  4.514851485148514],
 [0.636427076064201,
  0.6754016178648741,
  1.2093023255813953,
  8.250521920668058],
 [0.7687718843210998, 1.4301568154402895])

In [ ]:
# class_weights_dict = df_train['humour'].value_counts().to_dict()
class_weights_dict_h = { k : v for k,v in enumerate(class_weights_h)}
class_weights_dict_s = { k : v for k,v in enumerate(class_weights_s)}
class_weights_dict_o = { k : v for k,v in enumerate(class_weights_o)}
class_weights_dict_m = { k : v for k,v in enumerate(class_weights_m)}




In [ ]:
class_weights_dict_m

{0: 0.7687718843210998, 1: 1.4301568154402895}

In [ ]:


# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# for x in train_dataset.shuffle(50000).batch(256):

#   # print(x[0][0].decode('utf-8'))
#   # print((x[0]).numpy()[0].decode('utf-8'))
#   img_names = [c.decode('utf-8') for c in x[0].numpy()]
#   word_names = [c.decode('utf-8') for c in x[1].numpy()]

#   np_imgs, np_words = ret_img_word_embeddings(img_names, word_names)
#   np_h = tf.expand_dims(x[2],-1)


#   sample_weight = compute_sample_weight(class_weight=class_weights_dict, y=x[2])



#   with tf.GradientTape() as tape:

#       out = mtl_model([tf.squeeze(tf.convert_to_tensor(np_imgs), 1), tf.squeeze(tf.convert_to_tensor(np_words), 1)],training=True)

#       humour_loss = humour_loss_fn(np_h, out, sample_weight= sample_weight)
#       # sarcasm_loss = sarcasm_loss_fn(np_sarcasm, out[1])
#       # offensive_loss = offensive_loss_fn(np_offensive, out[2])
#       # motivational_loss = motivational_loss_fn(np_motivational, out[3])

#       loss = humour_loss # + sarcasm_loss + offensive_loss + motivational_loss



#   gradients = tape.gradient(loss, mtl_model.trainable_weights)
#   optimizer.apply_gradients(zip(gradients, mtl_model.trainable_weights))

#   humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out,axis=-1).numpy()), average='macro' )

#   print(loss, humour_metric)
#   print(out)
#   print(out.shape)
#   print(list(tf.argmax(out,axis=-1).numpy()))

#   # print((sample_weight))

#   # print(np_imgs.shape, np_words.shape, np_h.shape, tf.convert_to_tensor(np_words).dtype)
#   # print(len(img_names), len(word_names))
#   # print(x[2].shape)

#   break

In [ ]:
# df_word_cleaned['humour'] = df_word_cleaned['humour'].astype(np.uint8)
# df_word_cleaned['sarcasm'] = df_word_cleaned['sarcasm'].astype(np.uint8)
# df_word_cleaned['offensive'] = df_word_cleaned['offensive'].astype(np.uint8)
# df_word_cleaned['motivational'] = df_word_cleaned['motivational'].astype(np.uint8)

## MTL Model


In [ ]:
def get_mtl_model(model):

  # base_model.trainable = False

  # inp1 = base_model.layers[0].input
  # inp2 = base_model.layers[1].input

  # img_embedding = base_model.layers[-2].output
  # text_embedding = base_model.layers[-1].output

  # concat_x = tf.keras.layers.Concatenate()([img_embedding,text_embedding])
  # temp_layer = tf.keras.layers.Dense(256,activation='relu',name='first_hidden_layer')(concat_x)
  # temp_layer = tf.keras.layers.BatchNormalization(name='batch_norm_mtl')(temp_layer)


  # out1 = tf.keras.layers.Dense(4,activation='softmax',name='out_1')(temp_layer)
  # out2 = tf.keras.layers.Dense(4,activation='softmax',name='out_2')(temp_layer)
  # out3 = tf.keras.layers.Dense(4,activation='softmax',name='out_3')(temp_layer)
  # out4 = tf.keras.layers.Dense(1,name='out_4')(temp_layer)

  # mtl_model = tf.keras.Model(inputs=[inp1,inp2],outputs= [out1, out2, out3, out4])

  # return mtl_model

  model.trainable = False
  for layer in model.layers:
      layer.trainable =  False
  inp1 = model.layers[0].input
  inp2 = model.layers[1].input

  out1 = model.layers[-2].output
  out2 = model.layers[-1].output

  concat_x = tf.keras.layers.Concatenate()([out1,out2])
  temp_layer = tf.keras.layers.Dense(256,activation='tanh', kernel_regularizer='l2', name='dense_a')(concat_x)
  temp_layer = tf.keras.layers.BatchNormalization(name='batchnorm_a')(temp_layer)
  temp_layer = tf.keras.layers.Dense(256,activation='tanh', name='dense_b')(temp_layer)
  temp_layer = tf.keras.layers.BatchNormalization(name='batchnorm_b')(temp_layer)
  temp_layer = tf.keras.layers.Dense(256,activation='relu',name='dense_c')(temp_layer)
  temp_layer = tf.keras.layers.BatchNormalization(name='batchnorm_c')(temp_layer)


  pred_1 = tf.keras.layers.Dense(4,activation='softmax',name='out_1')(temp_layer)
  pred_2 = tf.keras.layers.Dense(4,activation='softmax',name='out_2')(temp_layer)
  pred_3 = tf.keras.layers.Dense(4,activation='softmax',name='out_3')(temp_layer)
  pred_4 = tf.keras.layers.Dense(1,name='out_4')(temp_layer)

  senti_model = tf.keras.Model(inputs=[inp1,inp2],outputs= [pred_1, pred_2, pred_3, pred_4])

  return senti_model

## Training

In [ ]:
# class_weights_humour = compute_class_weight('balanced', classes= np.unique((labels_temp_train[:,0])), y=labels_temp_train[:,0])
# class_weights_sarcasm = compute_class_weight('balanced', classes= np.unique((labels_temp_train[:,1])), y=labels_temp_train[:,1])
# class_weights_offensive = compute_class_weight('balanced', classes= np.unique((labels_temp_train[:,2])), y=labels_temp_train[:,2])
# class_weights_motivational = compute_class_weight('balanced', classes= np.unique((labels_temp_train[:,3])), y=labels_temp_train[:,3])

In [ ]:
# compute_sample_weight('balanced', y=labels_temp_train[:,0])

In [ ]:


# for x, y, z in train_dataset.batch(32):

#   np_humour = tf.expand_dims(z[:,0], axis=-1)
#   np_sarcasm = tf.expand_dims(z[:,1], axis=-1)
#   np_offensive = tf.expand_dims(z[:,2], axis=-1)
#   np_motivational = tf.expand_dims(z[:,3], axis=-1)

#   with tf.GradientTape() as tape:

#       out = mtl_model([x,y],training=True)

#       humour_loss = humour_loss_fn(np_humour, out[0], sample_weight= class_weights_humour)
#       sarcasm_loss = sarcasm_loss_fn(np_sarcasm, out[1])
#       offensive_loss = offensive_loss_fn(np_offensive, out[2])
#       motivational_loss = motivational_loss_fn(np_motivational, out[3])

#       loss = humour_loss + sarcasm_loss + offensive_loss + motivational_loss



#   gradients = tape.gradient(loss, mtl_model.trainable_weights)
#   optimizer.apply_gradients(zip(gradients, mtl_model.trainable_weights))

#   humour_metric = f1_score(list(tf.cast(np_humour[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[0],axis=-1).numpy()), average='macro' )
#   sarcasm_metric = f1_score(list(tf.cast(np_sarcasm[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[0],axis=-1).numpy()), average='macro' )
#   offensive_metric = f1_score(list(tf.cast(np_offensive[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[0],axis=-1).numpy()), average='macro' )
#   motivational_metric = f1_score(list(tf.cast(np_motivational[:,0], dtype=tf.int64).numpy()), list(tf.where(out[3]>=0.5,1,0)[:,0].numpy()), average='macro')
  
  

#   break

In [ ]:

# for x, y, z in train_dataset.batch(128):
#   # print(x.shape, y.shape, z.shape)
#   # print(type(x))

#   np_humour = z[:,0]
#   np_sarcasm = z[:,1]
#   np_offensive = z[:,2]
#   np_motivational = z[:,3]

#   print(np_humour.shape,np_sarcasm.shape,np_offensive.shape,np_motivational.shape)

#   print(list(np_motivational.numpy()))

#   # print(np_motivational.numpy().shape)

#   out = mtl_model([x,y],training=True)

#   print(list(out[3].numpy()[:,0]))

#   # print(out[3].numpy().shape)




#   prob = tf.where(out[3]>=0.5,1,0)

#   print(list(prob.numpy()[:,0]))

#   print(f1_score( list(np_motivational.numpy())  , list(prob.numpy()[:,0]) , average= 'macro' ) )

#   print()

#   # humour_loss = motivational_loss_fn(tf.cast(np.expand_dims(np_motivational,axis=-1), tf.int32), out[3])

#   # print(humour_loss)

#   break


In [ ]:
# humour_metric = tfa.metrics.F1Score(num_classes=4, average= 'macro')
# sarcasm_metric = tfa.metrics.F1Score(num_classes=4, average= 'macro')
# offensive_metric = tfa.metrics.F1Score(num_classes=4, average= 'macro')
# motivational_metric = tfa.metrics.F1Score(num_classes=1, average= 'macro')


# humour_metric_val = tfa.metrics.F1Score(num_classes=4, average= 'macro')
# sarcasm_metric_val = tfa.metrics.F1Score(num_classes=4, average= 'macro')
# offensive_metric_val = tfa.metrics.F1Score(num_classes=4, average= 'macro')

# humour_metric_test = tfa.metrics.F1Score(num_classes=4, average= 'macro')
# sarcasm_metric_test= tfa.metrics.F1Score(num_classes=4, average= 'macro')
# offensive_metric_test = tfa.metrics.F1Score(num_classes=4, average= 'macro')

humour_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
sarcasm_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
offensive_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
motivational_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)



In [ ]:
base_model = K.models.load_model('./drive/MyDrive/Wipro/Final/Base_Models/models/stanford_scratch_memotion_words_basic_model.hdf5')

In [ ]:
base_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_text (InputLayer)         [(None, 300)]        0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 4096)         16781312    input_image[0][0]                
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 4096)         1232896     input_text[0][0]                 
____________________________________________________________________________________________

In [ ]:
mtl_model = get_mtl_model(base_model)

In [ ]:
mtl_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 4096)]       0                                            
__________________________________________________________________________________________________
input_text (InputLayer)         [(None, 300)]        0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 4096)         16781312    input_image[0][0]                
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 4096)         1232896     input_text[0][0]                 
______________________________________________________________________________________________

In [ ]:
# mtl_model = get_mtl_model(base_model)

In [ ]:
def eval_metrics(dataset, model):


    # for a,b,c in val_dataset.batch(len(val_dataset)):


    #   np_humour_val = tf.expand_dims(c[:,0], axis=-1)
    #   np_sarcasm_val = tf.expand_dims(c[:,1], axis=-1)
    #   np_offensive_val = tf.expand_dims(c[:,2], axis=-1)
    #   np_motivational_val = tf.expand_dims(c[:,3], axis=-1)

    #   out_val = model.predict([a,b])

    #   humour_metric_val = f1_score(list(tf.cast(np_humour_val[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out_val[0],axis=-1).numpy()), average='macro' )
    #   sarcasm_metric_val = f1_score(list(tf.cast(np_sarcasm_val[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out_val[1],axis=-1).numpy()), average='macro' )
    #   offensive_metric_val = f1_score(list(tf.cast(np_offensive_val[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out_val[2],axis=-1).numpy()), average='macro' )
    #   motivational_metric_val = f1_score(list(tf.cast(np_motivational_val[:,0], dtype=tf.int64).numpy()), list(tf.where(out_val[3]>=0.5,1,0)[:,0].numpy()), average='macro')

    #   return humour_metric_val, sarcasm_metric_val, offensive_metric_val, motivational_metric_val



  for x in dataset.batch(len(dataset)):

    img_names = [c.decode('utf-8') for c in x[0].numpy()]
    word_names = [c.decode('utf-8') for c in x[1].numpy()]

    np_imgs, np_words = ret_img_word_embeddings(img_names, word_names)
    np_h = tf.expand_dims(x[2],-1)
    np_s = tf.expand_dims(x[3],-1)
    np_o = tf.expand_dims(x[4],-1)
    np_m = tf.expand_dims(x[5],-1)

    out = model.predict([tf.squeeze(tf.convert_to_tensor(np_imgs), 1), tf.squeeze(tf.convert_to_tensor(np_words), 1)])

    # humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out,axis=-1).numpy()), average='macro' )

    humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[0],axis=-1).numpy()), average='macro' )
    sarcasm_metric = f1_score(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[1],axis=-1).numpy()), average='macro' )
    offensive_metric = f1_score(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[2],axis=-1).numpy()), average='macro' )
    motivational_metric = f1_score(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()), list(tf.where(out[3]>=0.5,1,0)[:,0].numpy()), average='macro')

    print(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()))
    print(list(tf.argmax(out[0],axis=-1).numpy()))

    print(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()))
    print(list(tf.argmax(out[1],axis=-1).numpy()))
    
    print(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()))
    print(list(tf.argmax(out[2],axis=-1).numpy()))

    print(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()))
    print(list(tf.where(out[3]>=0.5,1,0)[:,0].numpy()))

    return humour_metric, sarcasm_metric, offensive_metric, motivational_metric


In [ ]:

# for x in train_dataset.shuffle(47000).batch(256):

#   img_names = [c.decode('utf-8') for c in x[0].numpy()]
#   word_names = [c.decode('utf-8') for c in x[1].numpy()]

#   np_imgs, np_words = ret_img_word_embeddings(img_names, word_names)
#   np_h = tf.expand_dims(x[2],-1)


#   sample_weight_h = compute_sample_weight(class_weight=class_weights_dict_h, y=x[2])
#   sample_weight_s = compute_sample_weight(class_weight=class_weights_dict_s, y=x[3])
#   sample_weight_o = compute_sample_weight(class_weight=class_weights_dict_o, y=x[4])
#   sample_weight_m = compute_sample_weight(class_weight=class_weights_dict_m, y=x[5])


#   print(sample_weight_h, sample_weight_s, sample_weight_o,sample_weight_m )
#   break

In [ ]:
def get_sample_weight(class_weight, y_train, motivation=False):

  sample_weight = np.ones(shape=(len(y_train),))
  sample_weight[y_train == 0] = class_weight[0]
  sample_weight[y_train == 1] = class_weight[1]

  if motivation:
    return sample_weight

  sample_weight[y_train == 2] = class_weight[2]
  sample_weight[y_train == 3] = class_weight[3]

  return sample_weight

In [ ]:
humour_metric_val_values = []
sarcasm_metric_val_values = []
offensive_metric_val_values = []
motivational_metric_val_values = []
average_metric_val_values = []

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
epochs = 75
best_val_acc = 0
cur_split = 1

for train_idx, test_idx in kfold.split(df_word_cleaned):
  # df_train = df_word_cleaned.iloc
  # print(train_idx.shape, test_idx.shape)
  mtl_model = get_mtl_model(base_model)
  
  print(f'\n\n----------------------------------------TRAINING SPLIT {cur_split}----------------------------------------')
  cur_split+=1

  df_train = df_word_cleaned.iloc[train_idx]
  df_val = df_word_cleaned.iloc[test_idx]

  # train_dataset  = tf.data.Dataset.from_tensor_slices(
  #     (
  #         df_train['image_name'].values,
  #         df_train['word'].values,
  #         df_train['humour'].values,
  #         df_train['sarcasm'].values,
  #         df_train['offensive'].values,
  #         df_train['motivational'].values,
  #     )  
  #   )



  train_dataset = ret_dataset(df_train)
  val_dataset = ret_dataset(df_val)


  for epoch in range(epochs):

    print(f'Epoch {epoch+1}/{epochs} ',end=' ')

    losses = []


    for x in train_dataset.shuffle(38000).batch(64):

      img_names = [c.decode('utf-8') for c in x[0].numpy()]
      word_names = [c.decode('utf-8') for c in x[1].numpy()]

      np_imgs, np_words = ret_img_word_embeddings(img_names, word_names)
      np_h = tf.expand_dims(x[2],-1)
      np_s = tf.expand_dims(x[3],-1)
      np_o = tf.expand_dims(x[4],-1)
      np_m = tf.expand_dims(x[5],-1)


      # sample_weight_h = compute_sample_weight(class_weight=class_weights_dict_h, y=x[2])
      # sample_weight_s = compute_sample_weight(class_weight=class_weights_dict_s, y=x[3])
      # sample_weight_o = compute_sample_weight(class_weight=class_weights_dict_o, y=x[4])
      # sample_weight_m = compute_sample_weight(class_weight=class_weights_dict_m, y=x[5])
      sample_weight_h = get_sample_weight(class_weights_dict_h, x[2])
      sample_weight_s = get_sample_weight(class_weights_dict_s, x[3])
      sample_weight_o = get_sample_weight(class_weights_dict_o, x[4])
      sample_weight_m = get_sample_weight(class_weights_dict_m, x[5], True)



      with tf.GradientTape() as tape:

          out = mtl_model([tf.squeeze(tf.convert_to_tensor(np_imgs), 1), tf.squeeze(tf.convert_to_tensor(np_words), 1)],training=True)

          humour_loss = humour_loss_fn(np_h, out[0], sample_weight= sample_weight_h)
          sarcasm_loss = sarcasm_loss_fn(np_s, out[1], sample_weight= sample_weight_s)
          offensive_loss = offensive_loss_fn(np_o, out[2], sample_weight= sample_weight_o)
          motivational_loss = motivational_loss_fn(np_m, out[3], sample_weight= sample_weight_m)

          loss = humour_loss  + sarcasm_loss + offensive_loss + motivational_loss

          losses.append(loss.numpy())

      gradients = tape.gradient(loss, mtl_model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, mtl_model.trainable_weights))

    humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[0],axis=-1).numpy()), average='macro' )
    sarcasm_metric = f1_score(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[1],axis=-1).numpy()), average='macro' )
    offensive_metric = f1_score(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[2],axis=-1).numpy()), average='macro' )
    motivational_metric = f1_score(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()), list(tf.where(out[3]>=0.5,1,0)[:,0].numpy()), average='macro')
    average_metric = (humour_metric+ sarcasm_metric+ offensive_metric+ motivational_metric)/4


    
    print(f'Humour F1 score {humour_metric} ', end=' ')
    print(f'Sarcasm F1 score {sarcasm_metric} ---- ', end=' ')
    print(f'Offensive F1 score {offensive_metric} ---- ', end=' ')
    print(f'Motivational F1 score {motivational_metric}', end=' ')
    print(f'Avg F1 score {average_metric}', end=' ')
    print(f'Loss {sum(losses)/len(losses)}',)
    print()


  humour_metric_val, sarcasm_metric_val, offensive_metric_val, motivational_metric_val = eval_metrics(val_dataset, mtl_model)
  average_metric_val = (humour_metric_val+ sarcasm_metric_val+ offensive_metric_val+ motivational_metric_val)/4
  # humour_metric_val = eval_metrics(val_dataset, mtl_model)

  print()

  print(f'Val Humour F1 score {humour_metric_val}', end=' ')
  print(f'Val Sarcasm F1 score {sarcasm_metric_val} ---- ', end=' ')
  print(f'Val Offensive F1 score {offensive_metric_val} ---- ', end=' ')
  print(f'Val Motivational F1 score {motivational_metric_val}')
  print(f'Val Avg F1 score {average_metric_val}', end=' ')

  humour_metric_val_values.append(humour_metric_val)
  sarcasm_metric_val_values.append(sarcasm_metric_val)
  offensive_metric_val_values.append(offensive_metric_val)
  motivational_metric_val_values.append(motivational_metric_val)
  average_metric_val_values.append(average_metric_val)

  # if average_metric_val > best_val_acc:
  #   best_val_acc = average_metric_val
  #   print('\n!!! Saved Model !!!\n')
  #   K.models.save_model(mtl_model, './scratch_flickr30k_memotion_words.hdf5')

    # humour_metric_test, sarcasm_metric_test, offensive_metric_test, motivational_metric_test = eval_metrics(test_dataset, mtl_model)
    # average_metric_test = (humour_metric_test+ sarcasm_metric_test+ offensive_metric_test+ motivational_metric_test)/4

    # print()

    # print(f'Test Humour F1 score {humour_metric_test}')
    # print(f'Test Sarcasm F1 score {sarcasm_metric_test}')
    # print(f'Test Offensive F1 score {offensive_metric_test}')
    # print(f'Test Motivational F1 score {motivational_metric_test}')
    # print(f'Test Avg F1 score {average_metric_test}')
    
    # print()

  # break



----------------------------------------TRAINING SPLIT 1----------------------------------------
Epoch 1/75  Humour F1 score 0.18727598566308243  Sarcasm F1 score 0.2287851037851038 ----  Offensive F1 score 0.22990995311167944 ----  Motivational F1 score 0.37037037037037035 Avg F1 score 0.254085353232559 Loss 5.4112486051266755

Epoch 2/75  Humour F1 score 0.1920300751879699  Sarcasm F1 score 0.15897129186602868 ----  Offensive F1 score 0.2629683404139926 ----  Motivational F1 score 0.3780487804878049 Avg F1 score 0.24800462198894901 Loss 4.974172383898627

Epoch 3/75  Humour F1 score 0.265250626566416  Sarcasm F1 score 0.1888223772414949 ----  Offensive F1 score 0.14648507678933642 ----  Motivational F1 score 0.5277777777777777 Avg F1 score 0.2820839645937563 Loss 4.903189335866481

Epoch 4/75  Humour F1 score 0.26329022988505746  Sarcasm F1 score 0.11266447368421054 ----  Offensive F1 score 0.1981737012987013 ----  Motivational F1 score 0.42696629213483145 Avg F1 score 0.2502736742

In [ ]:
humour_metric_val_values = []
sarcasm_metric_val_values = []
offensive_metric_val_values = []
motivational_metric_val_values = []
average_metric_val_values = []

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4)
epochs = 75
best_val_acc = 0
cur_split = 1

for train_idx, test_idx in kfold.split(df_word_cleaned):
  # df_train = df_word_cleaned.iloc
  # print(train_idx.shape, test_idx.shape)
  mtl_model = get_mtl_model(base_model)
  
  print(f'\n\n----------------------------------------TRAINING SPLIT {cur_split}----------------------------------------')
  cur_split+=1

  df_train = df_word_cleaned.iloc[train_idx]
  df_val = df_word_cleaned.iloc[test_idx]

  # train_dataset  = tf.data.Dataset.from_tensor_slices(
  #     (
  #         df_train['image_name'].values,
  #         df_train['word'].values,
  #         df_train['humour'].values,
  #         df_train['sarcasm'].values,
  #         df_train['offensive'].values,
  #         df_train['motivational'].values,
  #     )  
  #   )



  train_dataset = ret_dataset(df_train)
  val_dataset = ret_dataset(df_val)


  for epoch in range(epochs):

    print(f'Epoch {epoch+1}/{epochs} ',end=' ')

    losses = []


    for x in train_dataset.shuffle(38000).batch(64):

      img_names = [c.decode('utf-8') for c in x[0].numpy()]
      word_names = [c.decode('utf-8') for c in x[1].numpy()]

      np_imgs, np_words = ret_img_word_embeddings(img_names, word_names)
      np_h = tf.expand_dims(x[2],-1)
      np_s = tf.expand_dims(x[3],-1)
      np_o = tf.expand_dims(x[4],-1)
      np_m = tf.expand_dims(x[5],-1)


      # sample_weight_h = compute_sample_weight(class_weight=class_weights_dict_h, y=x[2])
      # sample_weight_s = compute_sample_weight(class_weight=class_weights_dict_s, y=x[3])
      # sample_weight_o = compute_sample_weight(class_weight=class_weights_dict_o, y=x[4])
      # sample_weight_m = compute_sample_weight(class_weight=class_weights_dict_m, y=x[5])
      sample_weight_h = get_sample_weight(class_weights_dict_h, x[2])
      sample_weight_s = get_sample_weight(class_weights_dict_s, x[3])
      sample_weight_o = get_sample_weight(class_weights_dict_o, x[4])
      sample_weight_m = get_sample_weight(class_weights_dict_m, x[5], True)



      with tf.GradientTape() as tape:

          out = mtl_model([tf.squeeze(tf.convert_to_tensor(np_imgs), 1), tf.squeeze(tf.convert_to_tensor(np_words), 1)],training=True)

          humour_loss = humour_loss_fn(np_h, out[0], sample_weight= sample_weight_h)
          sarcasm_loss = sarcasm_loss_fn(np_s, out[1], sample_weight= sample_weight_s)
          offensive_loss = offensive_loss_fn(np_o, out[2], sample_weight= sample_weight_o)
          motivational_loss = motivational_loss_fn(np_m, out[3], sample_weight= sample_weight_m)

          loss = humour_loss  + sarcasm_loss + offensive_loss + motivational_loss

          losses.append(loss.numpy())

      gradients = tape.gradient(loss, mtl_model.trainable_weights)
      optimizer.apply_gradients(zip(gradients, mtl_model.trainable_weights))

    humour_metric = f1_score(list(tf.cast(np_h[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[0],axis=-1).numpy()), average='macro' )
    sarcasm_metric = f1_score(list(tf.cast(np_s[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[1],axis=-1).numpy()), average='macro' )
    offensive_metric = f1_score(list(tf.cast(np_o[:,0], dtype=tf.int64).numpy()), list(tf.argmax(out[2],axis=-1).numpy()), average='macro' )
    motivational_metric = f1_score(list(tf.cast(np_m[:,0], dtype=tf.int64).numpy()), list(tf.where(out[3]>=0.5,1,0)[:,0].numpy()), average='macro')
    average_metric = (humour_metric+ sarcasm_metric+ offensive_metric+ motivational_metric)/4


    
    print(f'Humour F1 score {humour_metric} ', end=' ')
    print(f'Sarcasm F1 score {sarcasm_metric} ---- ', end=' ')
    print(f'Offensive F1 score {offensive_metric} ---- ', end=' ')
    print(f'Motivational F1 score {motivational_metric}', end=' ')
    print(f'Avg F1 score {average_metric}', end=' ')
    print(f'Loss {sum(losses)/len(losses)}',)
    print()


  humour_metric_val, sarcasm_metric_val, offensive_metric_val, motivational_metric_val = eval_metrics(val_dataset, mtl_model)
  average_metric_val = (humour_metric_val+ sarcasm_metric_val+ offensive_metric_val+ motivational_metric_val)/4
  # humour_metric_val = eval_metrics(val_dataset, mtl_model)

  print()

  print(f'Val Humour F1 score {humour_metric_val}', end=' ')
  print(f'Val Sarcasm F1 score {sarcasm_metric_val} ---- ', end=' ')
  print(f'Val Offensive F1 score {offensive_metric_val} ---- ', end=' ')
  print(f'Val Motivational F1 score {motivational_metric_val}')
  print(f'Val Avg F1 score {average_metric_val}', end=' ')

  humour_metric_val_values.append(humour_metric_val)
  sarcasm_metric_val_values.append(sarcasm_metric_val)
  offensive_metric_val_values.append(offensive_metric_val)
  motivational_metric_val_values.append(motivational_metric_val)
  average_metric_val_values.append(average_metric_val)

  # if average_metric_val > best_val_acc:
  #   best_val_acc = average_metric_val
  #   print('\n!!! Saved Model !!!\n')
  #   K.models.save_model(mtl_model, './scratch_flickr30k_memotion_words.hdf5')

    # humour_metric_test, sarcasm_metric_test, offensive_metric_test, motivational_metric_test = eval_metrics(test_dataset, mtl_model)
    # average_metric_test = (humour_metric_test+ sarcasm_metric_test+ offensive_metric_test+ motivational_metric_test)/4

    # print()

    # print(f'Test Humour F1 score {humour_metric_test}')
    # print(f'Test Sarcasm F1 score {sarcasm_metric_test}')
    # print(f'Test Offensive F1 score {offensive_metric_test}')
    # print(f'Test Motivational F1 score {motivational_metric_test}')
    # print(f'Test Avg F1 score {average_metric_test}')
    
    # print()

  # break



----------------------------------------TRAINING SPLIT 1----------------------------------------
Epoch 1/75  Humour F1 score 0.19263651002781437  Sarcasm F1 score 0.23157894736842105 ----  Offensive F1 score 0.22060931899641578 ----  Motivational F1 score 0.4670604270786006 Avg F1 score 0.27797130086781296 Loss 5.39955280442262

Epoch 2/75  Humour F1 score 0.24060739887352792  Sarcasm F1 score 0.16964912280701755 ----  Offensive F1 score 0.19882034632034634 ----  Motivational F1 score 0.44363636363636366 Avg F1 score 0.26317830790931385 Loss 5.060876944414882

Epoch 3/75  Humour F1 score 0.12352941176470589  Sarcasm F1 score 0.32272686220054636 ----  Offensive F1 score 0.309184285500075 ----  Motivational F1 score 0.40064102564102566 Avg F1 score 0.28902039627658827 Loss 4.956724617975759

Epoch 4/75  Humour F1 score 0.2803480087687249  Sarcasm F1 score 0.22897727272727272 ----  Offensive F1 score 0.2628205128205128 ----  Motivational F1 score 0.4204545454545454 Avg F1 score 0.298150

In [ ]:
sum(average_metric_val_values)/len(average_metric_val_values)

0.3241117841837723

In [ ]:
K.models.save_model(mtl_model, './drive/MyDrive/Wipro/Final/Task_C/kfold/stanford/models/memotion_words_stanford_memotion_taskc_best_split.hdf5')